# Non Linear Probabilistic Component Analysis

There is to find the posterior parameters of the random variable Z, assuming that the distribution is normal.Computed distribution will be the hidden representation of the data.

In [13]:
import numpy as np
import inferpy as inf
import tensorflow as tf

In [4]:
# number of components
k = 1
# size of the hidden layer in the NN
dh0 = 100
# dimensionality of the data
dx = 2
# number of observations (dataset size)
N = 1000

In [20]:
data = np.random.normal(2, 3, size=(N, dx))
x_train, x_test = np.split(data, [int(0.6*data.shape[0])])

In [2]:
def decoder(z, dh0, dx):
    h0 = tf.layers.dense(inputs=z, units=dh0, activation=tf.nn.relu)
    return tf.layers.dense(inputs=h0, units=2 * dx)


@inf.probmodel
def nlpca(k, d0, dx, decoder):

    with inf.datamodel():
        z = inf.Normal(loc=tf.ones([k]) * 0.5, scale=1. , name="z")    # shape = [N,k]
        output = decoder(z, dh0, dx)
        x_loc = output[:, :dx]
        x_scale = tf.nn.softmax(output[:,dx:])
        x = inf.Normal(x_loc, x_scale, name="x")   # shape = [N,d]

The "with" statement open context which is scheme for probabilistic model.<br>
"inf.Normal(.." declare random variable with Normal distribution. In this situation "z" represent latent variable<br>
loc - mean, scale - std(sigma)<br>


In [9]:
# create an instance of the model
m = nlpca(k,dh0,dx, decoder)

# Sample from priors
# samples = m.sample()

In [11]:
@inf.probmodel
def qmodel(k):
    with inf.datamodel():
        qz_loc = inf.Parameter(initial_value=tf.ones([k]) * 0.5, name="qz_loc")
        qz_scale = tf.math.softplus(inf.Parameter(initial_value=tf.ones([k]), name="qz_scale"))

        qz = inf.Normal(loc=qz_loc, scale=qz_scale, name="z")

In [24]:
# set the inference algorithm
VI = inf.inference.VI(qmodel(k), epochs=5000)

# learn the parameters
m.fit({"x": x_train}, VI)

W0813 01:07:46.860688 140063588685632 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/inferpy/contextmanager/evidence.py:15: Variable.load (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.



 0 epochs	 33345.3125....................
 200 epochs	 23354.64453125....................
 400 epochs	 23172.18359375....................
 600 epochs	 22649.2890625....................
 800 epochs	 21934.64453125....................
 1000 epochs	 21032.359375....................
 1200 epochs	 21174.751953125....................
 1400 epochs	 18778.951171875....................
 1600 epochs	 18305.732421875....................
 1800 epochs	 16881.64453125....................
 2000 epochs	 15717.505859375....................
 2200 epochs	 15175.4140625....................
 2400 epochs	 14413.2216796875....................
 2600 epochs	 13760.638671875....................
 2800 epochs	 13932.9501953125....................
 3000 epochs	 12992.833984375....................
 3200 epochs	 12480.5400390625....................
 3400 epochs	 11964.8662109375....................
 3600 epochs	 11778.8916015625....................
 3800 epochs	 11286.3154296875....................
 4000 epochs	 10

In [27]:
#extract the hidden representation
hidden_encoding = m.posterior("z")
print(hidden_encoding.sample())

[[ 2.5608571e+00]
 [-7.5036186e-01]
 [ 2.9274994e-01]
 [ 3.4922788e-01]
 [ 3.7561834e-01]
 [-7.4902272e-01]
 [-7.3215258e-01]
 [-2.1402910e-02]
 [ 1.9562047e+00]
 [-1.9743977e+00]
 [-4.2268997e-01]
 [ 1.1951679e+00]
 [ 2.1412849e+00]
 [-6.5323782e-01]
 [ 4.9823228e-01]
 [ 4.8240180e+00]
 [-1.2480864e-01]
 [ 1.6997917e+00]
 [ 3.8510388e-01]
 [-8.1193626e-01]
 [-8.3708817e-01]
 [-3.4612942e-01]
 [ 2.0920236e+00]
 [-2.0427069e-01]
 [ 1.8048070e-01]
 [ 1.6492764e+00]
 [-5.5041051e-01]
 [ 2.5474696e+00]
 [ 2.2745612e+00]
 [-4.2997038e-01]
 [ 2.2342065e-01]
 [ 2.7849603e-01]
 [-3.7264347e+00]
 [ 5.3229868e-01]
 [-3.5610902e-01]
 [ 1.4390638e+00]
 [ 1.0143241e+00]
 [ 7.3050952e-01]
 [ 1.3974798e+00]
 [-2.6903814e-01]
 [ 4.4582370e-01]
 [-3.8372478e-01]
 [-9.4691467e-01]
 [ 1.2738960e+00]
 [ 1.4232700e-01]
 [ 1.5388479e+00]
 [ 2.4554074e+00]
 [ 5.4973567e-01]
 [-1.5396736e+00]
 [-1.1597843e+00]
 [-3.4187131e-02]
 [ 4.7143534e-01]
 [ 2.6354823e-01]
 [ 1.6238374e+00]
 [-1.9617748e-01]
 [-9.22283